In [1]:
import os

load_directory = '../models/2025-01-12_重現paper實驗_resample至31596'
vocab_path = os.path.join(load_directory, "vocab.txt")

# 確認文件是否存在
if os.path.isfile(vocab_path):
    print(f"詞彙文件存在於: {vocab_path}")
else:
    print("詞彙文件不存在，請檢查保存目錄中的 vocab 文件是否正確命名為 vocab.txt")

詞彙文件存在於: ../models/2025-01-12_重現paper實驗_resample至31596\vocab.txt


In [2]:
from transformers import BertTokenizer
import os

load_directory = '../models/2025-01-12_重現paper實驗_resample至31596'
vocab_path = os.path.join(load_directory, "vocab.txt")

# 顯式指定詞彙文件路徑
tokenizer = BertTokenizer(vocab_file=vocab_path)
print("Tokenizer 加載完成！")

Tokenizer 加載完成！


In [3]:
from transformers import BertTokenizer, TFBertModel

# 設定保存的目錄
load_directory = '../models/2025-01-12_重現paper實驗_resample至31596'

# 加載保存的模型和 tokenizer
model = TFBertModel.from_pretrained(load_directory)

print("模型和 Tokenizer 加載完成！")

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


模型和 Tokenizer 加載完成！


In [5]:
import pandas as pd
df = pd.read_csv("../data/open_source_8500_preprocess_resample_31596.csv")
df_test = df.sample(n=1000, random_state=42).reset_index(drop=True)  # 隨機抽樣 1000 筆資料並重設索引
df_test

,Description,Assignment group,cleanText,Assignment_group_label,Inbound Message,Label
0,refer email sent,32,refer email sent,32,refer email sent,32
1,employee took vsp therefore remove ukxtqfda qv...,46,employee took vsp therefore remove ukxtqfda qv...,46,employee took vsp therefore remove ukxtqfda qv...,46
2,erp is slow xd internet is running fine xd oth...,31,erp slow internet running fine applications ou...,31,erp slow internet running fine applications ou...,31
3,xd job job failed in job scheduler at,2,job job failed job scheduler,2,job job failed job scheduler,2
4,called in for an issue where the internet is c...,31,called issue internet completely unable access...,31,called issue internet completely unable access...,31
...,...,...,...,...,...,...
995,probleme mit archiving tool,17,probleme mit archiving tool,17,probleme mit archiving tool,17
996,please install project back on the clients pc,23,please install project back clients,23,please install project back clients,23
997,performance problems with erp ieas and eu tool...,18,performance problems erp ieas tool especially ...,18,performance problems erp ieas tool especially ...,18
998,i have several company center accounts that ar...,14,several company center accounts pulling inform...,14,several company center accounts pulling inform...,14


In [6]:
from torch.utils.data import DataLoader, Dataset
class message_Dataset(Dataset):

  def __init__(self, messages, targets, tokenizer, max_length):
    self.messages=messages
    self.targets=targets
    self.tokenizer=tokenizer
    self.max_length=max_length

  def __len__(self):
    return len(self.messages)

  def __getitem__(self,item):
    message = str(self.messages[item])
    targets = self.targets[item]

    encoding = self.tokenizer.encode_plus(
        message,
        max_length=Max_length,
        add_special_tokens=True,
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt'
       )
           
    return {
        'message_text':message,
        'input_ids':encoding['input_ids'].flatten(),
        'attention_mask':encoding['attention_mask'].flatten(),
        'targets' : torch.tensor(targets,dtype=torch.long)
    }

In [7]:
batch_size=8
Max_length= 35
def data_loader(df,tokenizer, max_length, batch):
  ds=message_Dataset(
      messages=df['Inbound Message'].to_numpy(),
      targets=df['Label'].to_numpy(),
      tokenizer=tokenizer,
      max_length=Max_length
  )

  return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=0
  )

test_DataLoader=data_loader(df_test, tokenizer,Max_length,batch_size)

In [8]:
def get_predictions(model, data_loader):
  model = model.eval()
  messages = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:
      texts = d["message_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      messages.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return messages, predictions, prediction_probs, real_values

In [9]:
messages, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_DataLoader
)

AttributeError: 'TFBertModel' object has no attribute 'eval'

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
import numpy as np

# 假設 y_true 是真實標籤，y_probas 是多類別的預測概率
classes = np.unique(y_true)  # 獲取所有類別
y_true_binarized = label_binarize(y_true, classes=classes)  # 二值化真實標籤


plt.figure(figsize=(10, 8))  # 調整圖表大小
for i, class_label in enumerate(classes):
    fpr, tpr, _ = roc_curve(y_true_binarized[:, i], y_probas[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Class {class_label} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multiclass Classification')

# 將圖例放在圖表右側外部
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), fontsize='small')
plt.tight_layout()  # 自動調整圖表和標籤的布局
plt.savefig('roc_curve_multiclass_adjusted.pdf', bbox_inches='tight')
plt.show()